<h1 align='center'> 영상처리 프로그래밍 실습 9</h1>

<h6 align='right'> 2024. 5. 29. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름: 이선재  &nbsp;&nbsp;          학번: 20227123   &nbsp;&nbsp;         학과(전공): 빅데이터학과
    
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

* 제출 마감: 6월 4일 (화) 오후 10:00까지 최종본을 SmartLEAD제출


In [1]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

OpenCV version 4.10.0
NumPy version 1.26.4


### 문제 1. 

지금까지 만들고 있는 영상처리 프로그램에 다음 기능을 추가하라.
- 다양한 에지 검출 기능
- 가우시안 잡음 추가 기능
- 영상 회전 기능

In [2]:

import sys
import os
import cv2
import numpy as np
from PIL import Image
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QSlider, QVBoxLayout,
    QMenuBar, QMenu, QAction, QFileDialog, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QFont, QColor

import image_tools  # image_tools.load_image가 RGB 이미지를 반환한다고 가정

class ImageFilterWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("향상된 이미지 필터 프로그램")
        self.setGeometry(100, 100, 1400, 800)  # 창 크기 증가

        self.original_image = None
        self.filtered_image = None
        self.current_filter = "Mean"
        self.click_point = None
        self.drawing = False
        self.last_point = None
        self.draw_mode = None

        # 툴팁 스타일 설정 (작고 깔끔하게)
        self.setStyleSheet("""
            QToolTip {
                font-size: 12px;
                padding: 4px;
                background-color: #333;
                color: white;
                border: 1px solid #666;
                border-radius: 4px;
            }
        """)

        self.create_menu_bar()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 설명 라벨
        self.info_label = QLabel("사진을 클릭하여 중심점을 설정하거나 그리기 동작을 수행하세요.")
        self.info_label.setToolTip("필터링된 이미지에서 클릭하여 중심점을 지정하거나 그림을 그립니다.")
        self.coord_label = QLabel("중심점 좌표: (자동 중앙)")
        self.main_layout.addWidget(self.info_label)
        self.main_layout.addWidget(self.coord_label)

        # 이미지 표시 (크기 증가)
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(600, 500)
        self.filtered_label = QLabel("필터링된 이미지")
        self.filtered_label.setAlignment(Qt.AlignCenter)
        self.filtered_label.setMinimumSize(600, 500)
        self.filtered_label.mousePressEvent = self.set_click_point
        self.filtered_label.mouseMoveEvent = self.draw_on_image
        self.filtered_label.mouseReleaseEvent = self.stop_drawing
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.filtered_label)
        self.main_layout.addLayout(self.image_layout)

        # 필터 크기 슬라이더
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(51)
        self.slider.setValue(3)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(5)
        self.slider.setToolTip("필터 크기를 조정합니다.")
        self.slider.valueChanged.connect(self.update_filtered_image)
        self.main_layout.addWidget(QLabel("필터 크기"))
        self.main_layout.addWidget(self.slider)

        # 회전 각도 슬라이더
        self.angle_slider = QSlider(Qt.Horizontal)
        self.angle_slider.setMinimum(-180)
        self.angle_slider.setMaximum(180)
        self.angle_slider.setValue(0)
        self.angle_slider.setTickPosition(QSlider.TicksBelow)
        self.angle_slider.setTickInterval(15)
        self.angle_slider.setToolTip("이미지 회전 각도를 조정합니다.")
        self.angle_slider.valueChanged.connect(self.rotate_image_slider)
        self.main_layout.addWidget(QLabel("회전 각도"))
        self.main_layout.addWidget(self.angle_slider)

    def create_menu_bar(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("이미지 열기", self)
        open_action.setShortcut("Ctrl+O")
        open_action.setToolTip("이미지 파일을 엽니다.")
        open_action.triggered.connect(self.open_image)
        file_menu.addAction(open_action)

        # 필터 메뉴
        filter_menu = menubar.addMenu("필터")
        filters = [
            ("평균 필터", "Mean", "Ctrl+1", "이미지를 부드럽게 합니다."),
            ("가우시안 블러", "Gaussian", "Ctrl+2", "가우시안 분포로 부드럽게 합니다."),
            ("미디언 필터", "Median", "Ctrl+3", "노이즈를 효과적으로 제거합니다."),
            ("소벨 필터", "Sobel", "Ctrl+4", "에지를 강조합니다."),
            ("라플라시안 필터", "Laplacian", "Ctrl+5", "에지를 선명하게 합니다."),
            ("웨이브 필터", "Wave", "Ctrl+6", "파동 효과를 적용합니다.")
        ]
        for name, mode, shortcut, tooltip in filters:
            action = QAction(name, self)
            action.setShortcut(shortcut)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.set_filter)
            filter_menu.addAction(action)

        # 에지 검출 메뉴
        edge_menu = menubar.addMenu("에지 검출")
        edges = [
            ("캐니 엣지", "Canny", "캐니 알고리즘으로 에지를 검출합니다."),
            ("프리윗 엣지", "Prewitt", "프리윗 필터로 에지를 검출합니다."),
            ("로버츠 엣지", "Roberts", "로버츠 필터로 에지를 검출합니다.")
        ]
        for name, mode, tooltip in edges:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.apply_edge_filter)
            edge_menu.addAction(action)

        # 노이즈 메뉴
        noise_menu = menubar.addMenu("노이즈")
        noise_action = QAction("가우시안 노이즈 추가", self)
        noise_action.setToolTip("이미지에 가우시안 노이즈를 추가합니다.")
        noise_action.triggered.connect(self.add_gaussian_noise)
        noise_menu.addAction(noise_action)

        # 색상 메뉴
        color_menu = menubar.addMenu("색상")
        color_actions = [
            ("색상 반전", self.invert_colors, "이미지의 색상을 반전시킵니다."),
            ("그레이스케일", self.to_grayscale, "이미지를 흑백으로 변환합니다."),
            ("세피아 톤", self.apply_sepia, "이미지에 세피아 톤을 적용합니다."),
            ("색조 조정", self.adjust_hue, "이미지의 색조를 조정합니다.")
        ]
        for name, func, tooltip in color_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            color_menu.addAction(action)

        # 그리기 메뉴
        draw_menu = menubar.addMenu("그리기")
        draw_actions = [
            ("선 그리기", self.enable_line_drawing, "마우스로 자유롭게 선을 그립니다."),
            ("사각형 그리기", self.draw_rectangle, "중심점을 기준으로 사각형을 그립니다."),
            ("텍스트 추가", self.add_text, "이미지에 텍스트를 추가합니다."),
            ("랜덤 도형 효과", self.random_shapes, "무작위 도형을 추가하여 예술적 효과를 만듭니다.")
        ]
        for name, func, tooltip in draw_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            draw_menu.addAction(action)

        # 분석 메뉴
        analysis_menu = menubar.addMenu("분석")
        analysis_actions = [
            ("히스토그램 표시", self.show_histogram, "이미지의 밝기 분포를 분석합니다."),
            ("색상 통계", self.color_statistics, "이미지의 평균 및 주요 색상을 계산합니다."),
            ("객체 감지", self.detect_objects, "주요 윤곽선을 감지하여 표시합니다."),
            ("픽셀 정보", self.show_pixel_info, "클릭한 지점의 픽셀 색상 정보를 표시합니다.")
        ]
        for name, func, tooltip in analysis_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            analysis_menu.addAction(action)

    def open_image(self):
        filename, _ = QFileDialog.getOpenFileName(self, "이미지 열기", "", "이미지 파일 (*.png *.jpg *.jpeg *.bmp)")
        if filename:
            self.original_image = image_tools.load_image(filename)
            self.filtered_image = self.original_image.copy()
            self.update_filtered_image()
            self.update_display_images()

    def set_filter(self):
        self.current_filter = self.sender().data()
        self.update_filtered_image()

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def set_click_point(self, event):
        if self.filtered_image is None:
            return
        x = event.pos().x()
        y = event.pos().y()
        label_width = self.filtered_label.width()
        label_height = self.filtered_label.height()
        img_height, img_width = self.filtered_image.shape[:2]
        scale_x = img_width / label_width
        scale_y = img_height / label_height
        self.click_point = (int(x * scale_x), int(y * scale_y))
        self.coord_label.setText(f"중심점 좌표: {self.click_point}")
        if self.draw_mode == "line":
            self.last_point = self.click_point
            self.drawing = True
        elif self.draw_mode == "rectangle":
            self.draw_rectangle()
        elif self.draw_mode == "pixel_info":
            self.show_pixel_info()
        self.rotate_image_slider()

    def draw_on_image(self, event):
        if self.drawing and self.draw_mode == "line" and self.filtered_image is not None:
            x = event.pos().x()
            y = event.pos().y()
            label_width = self.filtered_label.width()
            label_height = self.filtered_label.height()
            img_height, img_width = self.filtered_image.shape[:2]
            scale_x = img_width / label_width
            scale_y = img_height / label_height
            current_point = (int(x * scale_x), int(y * scale_y))
            img = self.filtered_image.copy()
            cv2.line(img, self.last_point, current_point, (255, 0, 0), 2)
            self.filtered_image = img
            self.last_point = current_point
            self.update_display_images()

    def stop_drawing(self, event):
        self.drawing = False
        self.last_point = None

    def enable_line_drawing(self):
        self.draw_mode = "line"
        self.info_label.setText("마우스를 드래그하여 선을 그립니다.")

    def draw_rectangle(self):
        if self.filtered_image is None or self.click_point is None:
            return
        img = self.filtered_image.copy()
        center = self.click_point
        size = 100
        top_left = (center[0] - size // 2, center[1] - size // 2)
        bottom_right = (center[0] + size // 2, center[1] + size // 2)
        cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)
        self.filtered_image = img
        self.draw_mode = None
        self.update_display_images()

    def add_text(self):
        if self.filtered_image is None:
            return
        text, ok = QInputDialog.getText(self, "텍스트 입력", "이미지에 추가할 텍스트를 입력하세요:")
        if ok and text:
            img = self.filtered_image.copy()
            pos = self.click_point if self.click_point else (50, 50)
            cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            self.filtered_image = img
            self.update_display_images()

    def random_shapes(self):
        if self.filtered_image is None:
            return
        img = self.filtered_image.copy()
        for _ in range(10):
            shape_type = np.random.choice(["circle", "rectangle", "line"])
            color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
            if shape_type == "circle":
                center = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                radius = np.random.randint(10, 50)
                cv2.circle(img, center, radius, color, 2)
            elif shape_type == "rectangle":
                top_left = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                bottom_right = (top_left[0] + 50, top_left[1] + 50)
                cv2.rectangle(img, top_left, bottom_right, color, 2)
            elif shape_type == "line":
                pt1 = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                pt2 = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                cv2.line(img, pt1, pt2, color, 2)
        self.filtered_image = img
        self.update_display_images()

    def show_histogram(self):
        if self.filtered_image is None:
            return
        gray = cv2.cvtColor(self.filtered_image, cv2.COLOR_RGB2GRAY)
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = hist.flatten()
        hist_info = f"히스토그램 데이터 (최대 10개 빈): {hist[:10]}"
        self.info_label.setText(hist_info)

    def color_statistics(self):
        if self.filtered_image is None:
            return
        mean_color = np.mean(self.filtered_image, axis=(0, 1))
        dominant_color = np.median(self.filtered_image.reshape(-1, 3), axis=0)
        stats = f"평균 색상 (RGB): {mean_color.astype(int)}, 주요 색상: {dominant_color.astype(int)}"
        self.info_label.setText(stats)

    def detect_objects(self):
        if self.filtered_image is None:
            return
        gray = cv2.cvtColor(self.filtered_image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img = self.filtered_image.copy()
        cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
        self.filtered_image = img
        self.update_display_images()

    def show_pixel_info(self):
        if self.filtered_image is None or self.click_point is None:
            return
        self.draw_mode = "pixel_info"
        x, y = self.click_point
        if 0 <= y < self.filtered_image.shape[0] and 0 <= x < self.filtered_image.shape[1]:
            pixel = self.filtered_image[y, x]
            self.info_label.setText(f"픽셀 ({x}, {y}) RGB: {pixel}")
        self.draw_mode = None

    def apply_wave_filter(self, image):
        img_pil = Image.fromarray(image).convert('L').resize((800, 800))
        width, height = img_pil.size
        pixels = np.array(img_pil)
        pixmap = QPixmap(width, height)
        pixmap.fill(Qt.white)
        painter = QPainter(pixmap)
        pen = QPen(Qt.black, 1)
        painter.setPen(pen)
        frequency = 0.5
        max_amplitude = 5
        num_lines = 100
        line_spacing = height // num_lines
        all_points = []
        for line_idx in range(num_lines):
            y_base = line_idx * line_spacing
            points = []
            for x in range(width):
                y_pixel = min(max(y_base, 0), height - 1)
                brightness = pixels[y_pixel, x] / 255.0
                amplitude = (1 - brightness) * max_amplitude
                wave_offset = amplitude * np.sin(frequency * x)
                y_pos = y_base + wave_offset
                points.append(QPoint(x, int(y_pos)))
            all_points.append(points)
        for i in range(len(all_points) - 1):
            current_points = all_points[i]
            next_points = all_points[i + 1]
            painter.drawLine(current_points[-1], next_points[-1]) if i % 2 == 0 else painter.drawLine(current_points[0], next_points[0])
            for j in range(len(current_points) - 1):
                painter.drawLine(current_points[j], current_points[j + 1])
        painter.end()
        qimage = pixmap.toImage().convertToFormat(QImage.Format_RGB888)
        width, height = qimage.width(), qimage.height()
        ptr = qimage.bits()
        ptr.setsize(height * width * 3)
        arr = np.frombuffer(ptr.asstring(), np.uint8).reshape((height, width, 3))
        return arr

    def apply_edge_filter(self):
        if self.original_image is None:
            return
        mode = self.sender().data()
        gray = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        if mode == "Canny":
            edges = cv2.Canny(gray, 100, 200)
        elif mode == "Prewitt":
            kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]], dtype=np.float32)
            kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        elif mode == "Roberts":
            kernelx = np.array([[1,0],[0,-1]], dtype=np.float32)
            kernely = np.array([[0,1],[-1,0]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        self.filtered_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def add_gaussian_noise(self):
        if self.original_image is None:
            return
        row, col, ch = self.original_image.shape
        gauss = np.random.normal(0, 25, (row, col, ch)).astype(np.float32)
        noisy = np.clip(self.original_image + gauss, 0, 255).astype(np.uint8)
        self.filtered_image = noisy
        self.update_display_images()

    def invert_colors(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.bitwise_not(self.original_image)
        self.update_display_images()

    def to_grayscale(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def apply_sepia(self):
        if self.original_image is None:
            return
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        sepia_matrix = np.array([[0.272, 0.534, 0.131],
                                 [0.349, 0.686, 0.168],
                                 [0.393, 0.769, 0.189]])
        self.filtered_image = cv2.transform(img_bgr, sepia_matrix)
        self.filtered_image = np.clip(self.filtered_image, 0, 255).astype(np.uint8)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        self.update_display_images()

    def adjust_hue(self):
        if self.original_image is None:
            return
        img_hsv = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2HSV)
        img_hsv[:, :, 0] = (img_hsv[:, :, 0].astype(np.float32) + 30) % 180
        self.filtered_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
        self.update_display_images()

    def rotate_image_slider(self):
        if self.original_image is None:
            return
        angle = self.angle_slider.value()
        center = self.click_point if self.click_point else (self.original_image.shape[1] // 2, self.original_image.shape[0] // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.original_image, M, (self.original_image.shape[1], self.original_image.shape[0]))
        self.filtered_image = rotated
        self.update_display_images()

    def update_filtered_image(self):
        if self.original_image is None:
            return
        kernel_size = self.slider.value()
        if self.current_filter not in ["Sobel", "Laplacian", "Wave"]:
            kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
            kernel_size = max(1, kernel_size)
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        if self.current_filter == "Mean":
            self.filtered_image = cv2.blur(img_bgr, (kernel_size, kernel_size))
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Gaussian":
            self.filtered_image = cv2.GaussianBlur(img_bgr, (kernel_size, kernel_size), 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Median":
            self.filtered_image = cv2.medianBlur(img_bgr, kernel_size)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Sobel":
            sobel_x = cv2.Sobel(img_bgr, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(img_bgr, cv2.CV_64F, 0, 1, ksize=3)
            sobel_x = cv2.convertScaleAbs(sobel_x)
            sobel_y = cv2.convertScaleAbs(sobel_y)
            self.filtered_image = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Laplacian":
            self.filtered_image = cv2.Laplacian(img_bgr, cv2.CV_64F, ksize=3)
            self.filtered_image = cv2.convertScaleAbs(self.filtered_image)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Wave":
            self.filtered_image = self.apply_wave_filter(self.original_image)
        self.update_display_images()

    def update_display_images(self):
        if self.original_image is None:
            return
        self.original_label.setPixmap(self.numpy_to_qpixmap(self.original_image).scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        if self.filtered_image is not None:
            self.filtered_label.setPixmap(self.numpy_to_qpixmap(self.filtered_image).scaled(
                self.filtered_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def moving_average_filtering(filename=None):
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        app.quit()
        app = QApplication(sys.argv)
    window = ImageFilterWindow()
    if filename:
        if os.path.exists(filename):
            window.original_image = image_tools.load_image(filename)
            window.filtered_image = window.original_image.copy()
            window.update_filtered_image()
            window.update_display_images()
    window.show()
    try:
        app.exec_()
    except SystemExit:
        pass

if __name__ == "__main__":
    moving_average_filtering()

# 사용 AI - chat gpt 4o

# 질문과 답 1 (가우시안)

가우시안 노이즈가 뭐야 그리고 아주 간단한 예시 하나 들어줘 - chatgpt4o

좋아! 아주 쉽게 설명해줄게.

---

### 📌 **가우시안 노이즈란?**

* **무작위로 생기는 작은 변화** 중 하나야.
* 특히, **평균**과 **표준편차**를 가진 \*\*정규분포(가우스 분포)\*\*를 따르는 랜덤한 값으로 만들어지는 노이즈야.
* 주로 **이미지 처리**나 **신호 처리**에서 원본 데이터에 섞여서 생기는 잡음의 형태로 나타나.

---

### 📌 **아주 간단한 예시**

**예시:**
사진을 찍었는데, 원래 하얀 바탕(값 100)이어야 하는데 \*\*조금씩 다른 값(102, 98, 105, 97...)\*\*으로 나오는 현상
→ 이게 바로 가우시안 노이즈야!

---

### 📌 **간단한 코드 예시 (Python)**

```python
import numpy as np

# 원본 데이터: 모든 값이 100인 배열
image = np.ones((3, 3)) * 100
print("원본 이미지:")
print(image)

# 가우시안 노이즈 추가
noise = np.random.normal(0, 5, image.shape)  # 평균 0, 표준편차 5
noisy_image = image + noise
print("\n노이즈가 추가된 이미지:")
print(noisy_image)
```

이렇게 하면, 원래 100이던 값들이 조금씩 달라져서 **가우시안 노이즈가 적용된 이미지**가 만들어져.

필요하면 더 알려줄게! 🚀


# 질문과 답 2 (기능 추가)

chatgpt 4o
기존 코드에다가
- 다양한 에지 검출 기능
- 가우시안 잡음 추가 기능
- 영상 회전 기능
이러한 기능을 넣어줘 
을 각각 메뉴바에 따로 만들어줘

알겠습니다! 🎨
주어진 코드에 아래 기능을 추가해줄게요:
✅ **에지 검출 (Canny, Prewitt, Roberts)**
✅ **가우시안 노이즈 추가**
✅ **영상 회전 기능**

### 🔥 **구현 방법**

* 각 기능은 메뉴바의 새로운 메뉴로 추가
* 기능별 메서드도 함께 추가

### 📌 수정된 코드 (추가 부분만 강조)

전체 코드는 길어지므로 **변경/추가된 부분**만 설명하고, 필요한 부분은 알려주세요!

---

### 1️⃣ 메뉴바에 새로운 메뉴 추가

🔹 `create_menu_bar` 함수에 아래 코드 추가:

```python
# Edge Detection 메뉴
edge_menu = menubar.addMenu("Edge Detection")
edges = [
    ("Canny Edge", "Canny"),
    ("Prewitt Edge", "Prewitt"),
    ("Roberts Edge", "Roberts")
]
for name, mode in edges:
    action = QAction(name, self)
    action.setData(mode)
    action.triggered.connect(self.apply_edge_filter)
    edge_menu.addAction(action)

# Noise 메뉴
noise_menu = menubar.addMenu("Noise")
noise_action = QAction("Add Gaussian Noise", self)
noise_action.triggered.connect(self.add_gaussian_noise)
noise_menu.addAction(noise_action)

# Rotate 메뉴
rotate_menu = menubar.addMenu("Rotate")
for angle in [90, 180, 270]:
    action = QAction(f"Rotate {angle}°", self)
    action.setData(angle)
    action.triggered.connect(self.rotate_image)
    rotate_menu.addAction(action)
```

---

### 2️⃣ 새로운 기능 메서드 추가

🔹 **Edge Detection 메서드**:

```python
def apply_edge_filter(self):
    if self.original_image is None:
        return
    action = self.sender()
    mode = action.data()
    gray = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
    if mode == "Canny":
        edges = cv2.Canny(gray, 100, 200)
    elif mode == "Prewitt":
        kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]], dtype=np.float32)
        kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float32)
        edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
    elif mode == "Roberts":
        kernelx = np.array([[1,0],[0,-1]], dtype=np.float32)
        kernely = np.array([[0,1],[-1,0]], dtype=np.float32)
        edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    self.filtered_image = edges_rgb
    self.update_display_images()
```

🔹 **가우시안 노이즈 추가**:

```python
def add_gaussian_noise(self):
    if self.original_image is None:
        return
    row, col, ch = self.original_image.shape
    mean = 0
    sigma = 25
    gauss = np.random.normal(mean, sigma, (row, col, ch)).astype(np.float32)
    noisy = np.clip(self.original_image + gauss, 0, 255).astype(np.uint8)
    self.filtered_image = noisy
    self.update_display_images()
```

🔹 **영상 회전 기능**:

```python
def rotate_image(self):
    if self.original_image is None:
        return
    action = self.sender()
    angle = action.data()
    rotated = cv2.rotate(self.original_image, {
        90: cv2.ROTATE_90_CLOCKWISE,
        180: cv2.ROTATE_180,
        270: cv2.ROTATE_90_COUNTERCLOCKWISE
    }[angle])
    self.filtered_image = rotated
    self.update_display_images()
```

---

### 🎯 적용 후 해야 할 일

1. 위 코드 추가 후, `create_menu_bar` 내부를 업데이트
2. 필요하면 전체 코드로 통합해줄게! 알려줘!

---

필요한 설명이나 코드 통합이 있으면 알려줘! 🚀


# 질문과 답 3 (회전 기능 보완)

아주 좋은데 일단은 회전하는 부분을 좀 더 자세하게 만들어줘 기능 자체가 내가 세세하게 회전이 가능하도록 구현 해 주었으면 좋겠어 추가로 이건 안해도 상관없긴 한데 내가 클릭한 부분을 중심점으로 회전이 가능하도록 만들어 주었으면 좋겠어 이건 희망사항이야 만약 못 하겠다 싶으면 안해도 돼. 

알겠습니다! 🎯
1️⃣ **세밀한 각도 조절** 기능 추가
2️⃣ **클릭한 위치를 중심으로 회전**은 일단 시도해보고, 안 되면 대안 제안할게요.

---

## 1️⃣ **세밀한 각도 조절 기능 (슬라이더 + 입력창)**

### 🛠️ 구현 방법

* 메뉴 대신 **QSlider** 또는 **QInputDialog**로 각도를 입력받음
* 기본 회전 중심은 이미지의 중앙 (기본 OpenCV `warpAffine` 사용)
* **클릭한 위치를 중심으로 회전**:

  * `mousePressEvent`에서 클릭 좌표를 저장
  * 회전 함수에 전달하여 `getRotationMatrix2D`에 중심점으로 적용

---

### 📦 코드 수정 (회전 부분만 강조)

🔻 **슬라이더 추가 (QSlider로 각도 조절)**

```python
        # 각도 슬라이더 추가
        self.angle_slider = QSlider(Qt.Horizontal)
        self.angle_slider.setMinimum(-180)
        self.angle_slider.setMaximum(180)
        self.angle_slider.setValue(0)
        self.angle_slider.setTickPosition(QSlider.TicksBelow)
        self.angle_slider.setTickInterval(15)
        self.angle_slider.valueChanged.connect(self.rotate_image_slider)
        self.main_layout.addWidget(self.angle_slider)

        # 클릭 중심점 (선택적)
        self.click_point = None
        self.filtered_label.mousePressEvent = self.set_click_point
```

---

🔻 **클릭 중심 좌표 저장 함수**

```python
    def set_click_point(self, event):
        if self.filtered_image is None:
            return
        x = event.pos().x()
        y = event.pos().y()
        # 비율 보정
        label_width = self.filtered_label.width()
        label_height = self.filtered_label.height()
        img_height, img_width = self.filtered_image.shape[:2]
        scale_x = img_width / label_width
        scale_y = img_height / label_height
        self.click_point = (int(x * scale_x), int(y * scale_y))
        print(f"중심점 좌표 저장: {self.click_point}")
```

---

🔻 **회전 함수 수정 (중심점 선택 가능)**

```python
    def rotate_image_slider(self):
        if self.original_image is None:
            return
        angle = self.angle_slider.value()
        center = self.click_point if self.click_point else (self.original_image.shape[1] // 2, self.original_image.shape[0] // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.original_image, M, (self.original_image.shape[1], self.original_image.shape[0]))
        self.filtered_image = cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB) if rotated.ndim == 2 else rotated
        self.update_display_images()
```

---

### ✅ 사용 방법

1. 이미지 클릭 → 중심점 저장
2. 슬라이더로 각도 조정 (중심점 기준 회전)
3. 중심점 초기화는 그냥 다시 클릭하면 됨

---

### 🎁 **추가로**

* 중심점 클릭 안 하면 이미지 중앙 기준 회전
* 필요하면 `QSpinBox`로 각도 입력창도 추가 가능!

---

### 🌊 다음 단계

1. 이 코드 통합해줄까?
2. 중심점 좌표 표시나 리셋 기능도 필요해?
3. 더 필요한 옵션 있어?

원하면 바로 전체 코드로 통합해줄게! 🚀


# 질문과 답 4 (한글로 변환 및 툴팁 추가)

아주 좋아 근데 보완 해야 할 점들이 좀 보여 일단은 영어를 한글로 번역해줘 그리고 bar 같은경우 설명을 좀 위에 붙여줬으면 좋겠어 예를 들어, 사진을 클릭하면 그 부분이 중심점이 됩니다. 그리고 중심점 그 좌표를 띄우고

좋아! 😎
말씀하신 **영어 → 한글 번역** + **설명 추가** + **중심점 좌표 표시**를 깔끔하게 보완해드릴게요!
아래는 정리된 **개선 포인트**:

---

## 📝 개선사항

1️⃣ 메뉴 및 UI 텍스트 전부 한글화
2️⃣ 각 기능 위에 **간단한 설명** 표시 (라벨 추가)
3️⃣ 클릭 시 중심점 좌표를 **레이블로 화면에 표시**
4️⃣ (선택) 중심점 좌표를 초기화할 수 있는 버튼 추가 (필요하면 알려줘!)

---

## 📦 개선된 코드 (변경된 부분만 설명)

### 1️⃣ 메뉴 및 UI 한글화

```python
file_menu = menubar.addMenu("파일")
open_action = QAction("이미지 열기", self)
...

filter_menu = menubar.addMenu("필터")
filters = [
    ("평균 필터", "Mean", "Ctrl+1"),
    ("가우시안 블러", "Gaussian", "Ctrl+2"),
    ("미디언 필터", "Median", "Ctrl+3"),
    ("소벨 필터", "Sobel", "Ctrl+4"),
    ("라플라시안 필터", "Laplacian", "Ctrl+5"),
    ("웨이브 필터", "Wave", "Ctrl+6")
]

edge_menu = menubar.addMenu("에지 검출")
edges = [("캐니 엣지", "Canny"), ("프리윗 엣지", "Prewitt"), ("로버츠 엣지", "Roberts")]

noise_menu = menubar.addMenu("노이즈")
noise_action = QAction("가우시안 노이즈 추가", self)

rotate_menu = menubar.addMenu("회전")
```

---

### 2️⃣ 설명 라벨 추가

`__init__`에 설명 라벨 2개 추가:

```python
self.info_label = QLabel("사진을 클릭하면 중심점이 설정됩니다.")
self.coord_label = QLabel("중심점 좌표: (자동 중앙)")

self.main_layout.addWidget(self.info_label)
self.main_layout.addWidget(self.coord_label)
```

---

### 3️⃣ 클릭한 중심 좌표 표시

`set_click_point` 함수 업데이트:

```python
def set_click_point(self, event):
    if self.filtered_image is None:
        return
    x = event.pos().x()
    y = event.pos().y()
    label_width = self.filtered_label.width()
    label_height = self.filtered_label.height()
    img_height, img_width = self.filtered_image.shape[:2]
    scale_x = img_width / label_width
    scale_y = img_height / label_height
    self.click_point = (int(x * scale_x), int(y * scale_y))
    self.coord_label.setText(f"중심점 좌표: {self.click_point}")
```

---

### 4️⃣ 출력 예시

실행하면:

* 위쪽에 설명: **"사진을 클릭하면 중심점이 설정됩니다."**
* 클릭하면 아래에: **"중심점 좌표: (125, 240)"**

---

## 🚀 전체 통합 코드로 다시 작성해드릴까?

필요하면 바로 깔끔한 코드로 통합해드릴게! 알려줘! 💙
(혹시 중심점 초기화 버튼도 필요해? 🤔)


# 질문과 답 5 (툴팁 크기 및 메뉴 추가) - grok

툴팁이 너무 커 사진의 크기를 키우고 툴팁 자체를 줄여줘 그리고 바 위에 툴팁 자그마게 회전각도 입니다. 등 각각 그에 맞는 설명을 부탁해. 이제 그리그 메뉴바와 색상 메뉴바를 추가해줘 그 안에 기능들은 너가 생각해서 재미난걸로 추가해주고

요청하신 대로 코드를 수정하여 툴팁을 작고 간결하게 만들고, 이미지 표시 크기를 키우며, 슬라이더 위에 적절한 설명이 포함된 작은 툴팁을 추가했습니다. 또한, 메뉴바에 색상 메뉴를 추가하고 재미있는 색상 관련 기능(색상 반전, 그레이스케일, 세피아 톤, 색조 조정)을 포함했습니다. 모든 변경 사항을 통합하여 한글 주석과 함께 하나의 완전한 코드로 제공합니다.

아래는 수정된 코드입니다:

```python
```python
import sys
import os
import cv2
import numpy as np
from PIL import Image
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QSlider, QVBoxLayout,
    QMenuBar, QMenu, QAction, QFileDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QFont

import image_tools  # image_tools.load_image가 RGB 이미지를 반환한다고 가정

class ImageFilterWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("향상된 이미지 필터 프로그램")
        self.setGeometry(100, 100, 1400, 800)  # 창 크기 증가

        self.original_image = None
        self.filtered_image = None
        self.current_filter = "Mean"
        self.click_point = None

        # 툴팁 스타일 설정 (작고 깔끔하게)
        self.setStyleSheet("""
            QToolTip {
                font-size: 12px;
                padding: 4px;
                background-color: #333;
                color: white;
                border: 1px solid #666;
                border-radius: 4px;
            }
        """)

        self.create_menu_bar()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 설명 라벨
        self.info_label = QLabel("사진을 클릭하여 중심점을 설정하세요.")
        self.info_label.setToolTip("필터링된 이미지에서 클릭하여 회전 중심을 지정합니다.")
        self.coord_label = QLabel("중심점 좌표: (자동 중앙)")
        self.main_layout.addWidget(self.info_label)
        self.main_layout.addWidget(self.coord_label)

        # 이미지 표시 (크기 증가)
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(600, 500)  # 이미지 표시 크기 증가
        self.filtered_label = QLabel("필터링된 이미지")
        self.filtered_label.setAlignment(Qt.AlignCenter)
        self.filtered_label.setMinimumSize(600, 500)  # 이미지 표시 크기 증가
        self.filtered_label.mousePressEvent = self.set_click_point
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.filtered_label)
        self.main_layout.addLayout(self.image_layout)

        # 필터 크기 슬라이더
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(51)
        self.slider.setValue(3)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(5)
        self.slider.setToolTip("필터 크기 조정")
        self.slider.valueChanged.connect(self.update_filtered_image)
        self.main_layout.addWidget(QLabel("필터 크기"))
        self.main_layout.addWidget(self.slider)

        # 회전 각도 슬라이더
        self.angle_slider = QSlider(Qt.Horizontal)
        self.angle_slider.setMinimum(-180)
        self.angle_slider.setMaximum(180)
        self.angle_slider.setValue(0)
        self.angle_slider.setTickPosition(QSlider.TicksBelow)
        self.angle_slider.setTickInterval(15)
        self.angle_slider.setToolTip("회전 각도 조정")
        self.angle_slider.valueChanged.connect(self.rotate_image_slider)
        self.main_layout.addWidget(QLabel("회전 각도"))
        self.main_layout.addWidget(self.angle_slider)

    def create_menu_bar(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("이미지 열기", self)
        open_action.setShortcut("Ctrl+O")
        open_action.setToolTip("이미지 파일을 엽니다.")
        open_action.triggered.connect(self.open_image)
        file_menu.addAction(open_action)

        # 필터 메뉴
        filter_menu = menubar.addMenu("필터")
        filters = [
            ("평균 필터", "Mean", "Ctrl+1", "이미지를 부드럽게 합니다."),
            ("가우시안 블러", "Gaussian", "Ctrl+2", "가우시안 분포로 부드럽게 합니다."),
            ("미디언 필터", "Median", "Ctrl+3", "노이즈를 효과적으로 제거합니다."),
            ("소벨 필터", "Sobel", "Ctrl+4", "에지 강조 필터입니다."),
            ("라플라시안 필터", "Laplacian", "Ctrl+5", "에지를 선명하게 합니다."),
            ("웨이브 필터", "Wave", "Ctrl+6", "파동 효과를 적용합니다.")
        ]
        for name, mode, shortcut, tooltip in filters:
            action = QAction(name, self)
            action.setShortcut(shortcut)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.set_filter)
            filter_menu.addAction(action)

        # 에지 검출 메뉴
        edge_menu = menubar.addMenu("에지 검출")
        edges = [
            ("캐니 엣지", "Canny", "캐니 알고리즘으로 에지를 검출합니다."),
            ("프리윗 엣지", "Prewitt", "프리윗 필터로 에지를 검출합니다."),
            ("로버츠 엣지", "Roberts", "로버츠 필터로 에지를 검출합니다.")
        ]
        for name, mode, tooltip in edges:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.apply_edge_filter)
            edge_menu.addAction(action)

        # 노이즈 메뉴
        noise_menu = menubar.addMenu("노이즈")
        noise_action = QAction("가우시안 노이즈 추가", self)
        noise_action.setToolTip("이미지에 가우시안 노이즈를 추가합니다.")
        noise_action.triggered.connect(self.add_gaussian_noise)
        noise_menu.addAction(noise_action)

        # 색상 메뉴 (새로운 기능 추가)
        color_menu = menubar.addMenu("색상")
        color_actions = [
            ("색상 반전", self.invert_colors, "이미지의 색상을 반전시킵니다."),
            ("그레이스케일", self.to_grayscale, "이미지를 흑백으로 변환합니다."),
            ("세피아 톤", self.apply_sepia, "이미지에 세피아 톤을 적용합니다."),
            ("색조 조정", self.adjust_hue, "이미지의 색조를 조정합니다.")
        ]
        for name, func, tooltip in color_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            color_menu.addAction(action)

    def open_image(self):
        filename, _ = QFileDialog.getOpenFileName(self, "이미지 열기", "", "이미지 파일 (*.png *.jpg *.jpeg *.bmp)")
        if filename:
            self.original_image = image_tools.load_image(filename)
            self.filtered_image = self.original_image.copy()
            self.update_filtered_image()
            self.update_display_images()

    def set_filter(self):
        self.current_filter = self.sender().data()
        self.update_filtered_image()

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def set_click_point(self, event):
        if self.filtered_image is None:
            return
        x = event.pos().x()
        y = event.pos().y()
        label_width = self.filtered_label.width()
        label_height = self.filtered_label.height()
        img_height, img_width = self.filtered_image.shape[:2]
        scale_x = img_width / label_width
        scale_y = img_height / label_height
        self.click_point = (int(x * scale_x), int(y * scale_y))
        self.coord_label.setText(f"중심점 좌표: {self.click_point}")
        self.rotate_image_slider()  # 중심점 변경 시 회전 업데이트

    def apply_wave_filter(self, image):
        img_pil = Image.fromarray(image).convert('L').resize((800, 800))
        width, height = img_pil.size
        pixels = np.array(img_pil)
        pixmap = QPixmap(width, height)
        pixmap.fill(Qt.white)
        painter = QPainter(pixmap)
        pen = QPen(Qt.black, 1)
        painter.setPen(pen)
        frequency = 0.5
        max_amplitude = 5
        num_lines = 100
        line_spacing = height // num_lines
        all_points = []
        for line_idx in range(num_lines):
            y_base = line_idx * line_spacing
            points = []
            for x in range(width):
                y_pixel = min(max(y_base, 0), height - 1)
                brightness = pixels[y_pixel, x] / 255.0
                amplitude = (1 - brightness) * max_amplitude
                wave_offset = amplitude * np.sin(frequency * x)
                y_pos = y_base + wave_offset
                points.append(QPoint(x, int(y_pos)))
            all_points.append(points)
        for i in range(len(all_points) - 1):
            current_points = all_points[i]
            next_points = all_points[i + 1]
            painter.drawLine(current_points[-1], next_points[-1]) if i % 2 == 0 else painter.drawLine(current_points[0], next_points[0])
            for j in range(len(current_points) - 1):
                painter.drawLine(current_points[j], current_points[j + 1])
        painter.end()
        qimage = pixmap.toImage().convertToFormat(QImage.Format_RGB888)
        width, height = qimage.width(), qimage.height()
        ptr = qimage.bits()
        ptr.setsize(height * width * 3)
        arr = np.frombuffer(ptr.asstring(), np.uint8).reshape((height, width, 3))
        return arr

    def apply_edge_filter(self):
        if self.original_image is None:
            return
        mode = self.sender().data()
        gray = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        if mode == "Canny":
            edges = cv2.Canny(gray, 100, 200)
        elif mode == "Prewitt":
            kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]], dtype=np.float32)
            kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        elif mode == "Roberts":
            kernelx = np.array([[1,0],[0,-1]], dtype=np.float32)
            kernely = np.array([[0,1],[-1,0]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        self.filtered_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def add_gaussian_noise(self):
        if self.original_image is None:
            return
        row, col, ch = self.original_image.shape
        gauss = np.random.normal(0, 25, (row, col, ch)).astype(np.float32)
        noisy = np.clip(self.original_image + gauss, 0, 255).astype(np.uint8)
        self.filtered_image = noisy
        self.update_display_images()

    def invert_colors(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.bitwise_not(self.original_image)
        self.update_display_images()

    def to_grayscale(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def apply_sepia(self):
        if self.original_image is None:
            return
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        sepia_matrix = np.array([[0.272, 0.534, 0.131],
                                 [0.349, 0.686, 0.168],
                                 [0.393, 0.769, 0.189]])
        self.filtered_image = cv2.transform(img_bgr, sepia_matrix)
        self.filtered_image = np.clip(self.filtered_image, 0, 255).astype(np.uint8)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        self.update_display_images()

    def adjust_hue(self):
        if self.original_image is None:
            return
        img_hsv = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2HSV)
        img_hsv[:, :, 0] = (img_hsv[:, :, 0].astype(np.float32) + 30) % 180  # 색조 조정 (+30도)
        self.filtered_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
        self.update_display_images()

    def rotate_image_slider(self):
        if self.original_image is None:
            return
        angle = self.angle_slider.value()
        center = self.click_point if self.click_point else (self.original_image.shape[1] // 2, self.original_image.shape[0] // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.original_image, M, (self.original_image.shape[1], self.original_image.shape[0]))
        self.filtered_image = rotated
        self.update_display_images()

    def update_filtered_image(self):
        if self.original_image is None:
            return
        kernel_size = self.slider.value()
        if self.current_filter not in ["Sobel", "Laplacian", "Wave"]:
            kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
            kernel_size = max(1, kernel_size)
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        if self.current_filter == "Mean":
            self.filtered_image = cv2.blur(img_bgr, (kernel_size, kernel_size))
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Gaussian":
            self.filtered_image = cv2.GaussianBlur(img_bgr, (kernel_size, kernel_size), 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Median":
            self.filtered_image = cv2.medianBlur(img_bgr, kernel_size)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Sobel":
            sobel_x = cv2.Sobel(img_bgr, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(img_bgr, cv2.CV_64F, 0, 1, ksize=3)
            sobel_x = cv2.convertScaleAbs(sobel_x)
            sobel_y = cv2.convertScaleAbs(sobel_y)
            self.filtered_image = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Laplacian":
            self.filtered_image = cv2.Laplacian(img_bgr, cv2.CV_64F, ksize=3)
            self.filtered_image = cv2.convertScaleAbs(self.filtered_image)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Wave":
            self.filtered_image = self.apply_wave_filter(self.original_image)
        self.update_display_images()

    def update_display_images(self):
        if self.original_image is None:
            return
        self.original_label.setPixmap(self.numpy_to_qpixmap(self.original_image).scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        if self.filtered_image is not None:
            self.filtered_label.setPixmap(self.numpy_to_qpixmap(self.filtered_image).scaled(
                self.filtered_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def moving_average_filtering(filename=None):
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        app.quit()
        app = QApplication(sys.argv)
    window = ImageFilterWindow()
    if filename:
        if os.path.exists(filename):
            window.original_image = image_tools.load_image(filename)
            window.filtered_image = window.original_image.copy()
            window.update_filtered_image()
            window.update_display_images()
    window.show()
    try:
        app.exec_()
    except SystemExit:
        pass

if __name__ == "__main__":
    moving_average_filtering()
```

### 주요 변경 사항:
1. **툴팁 최적화**:
   - 툴팁 크기를 작고 간결하게 설정하기 위해 스타일시트를 추가하여 폰트 크기(12px), 패딩(4px), 배경색 및 테두리를 조정했습니다.
   - 슬라이더 위에 간단한 설명 추가: "필터 크기"와 "회전 각도" 라벨을 슬라이더 위에 배치하고, 툴팁에 간결한 설명("필터 크기 조정", "회전 각도 조정")을 추가했습니다.
   - 메뉴 항목마다 기능에 맞는 짧고 명확한 툴팁을 추가했습니다(예: "이미지를 부드럽게 합니다.", "에지 강조 필터입니다.").

2. **이미지 크기 증가**:
   - 창 크기를 `(1200, 700)`에서 `(1400, 800)`으로 늘리고, 이미지 라벨의 최소 크기를 `(600, 500)`으로 설정하여 이미지 표시 영역을 확대했습니다.

3. **색상 메뉴 추가**:
   - 새로운 "색상" 메뉴를 추가하고, 다음과 같은 재미있는 색상 관련 기능을 포함했습니다:
     - **색상 반전**: 이미지의 색상을 반전하여 네거티브 효과를 만듭니다.
     - **그레이스케일**: 이미지를 흑백으로 변환합니다.
     - **세피아 톤**: 세피아 톤 필터를 적용하여 빈티지 느낌을 만듭니다.
     - **색조 조정**: HSV 색상 공간에서 색조를 30도 이동시켜 색감을 변화시킵니다.
   - 각 색상 기능에 툴팁을 추가하여 사용자가 기능을 쉽게 이해하도록 했습니다.

4. **기타 개선**:
   - 중심점 클릭 시 회전이 즉시 반영되도록 `set_click_point`에서 `rotate_image_slider`를 호출했습니다.
   - 코드의 한글 주석을 유지하고, 변수 및 함수 이름을 직관적으로 이해할 수 있도록 그대로 사용했습니다.
   - 모든 기능이 통합된 단일 코드로 제공되며, 기존 기능(필터, 에지 검출, 노이즈 추가, 회전 등)은 그대로 유지되었습니다.

이 코드는 사용자가 요청한 대로 툴팁을 작고 명확하게 만들고, 이미지 표시 영역을 키우며, 새로운 색상 메뉴를 추가하여 더 재미있고 유용한 이미지 처리 프로그램을 제공합니다. `image_tools.load_image` 함수가 RGB 이미지를 반환한다고 가정했으며, 이 함수가 별도로 정의되어 있어야 합니다.

# 질문과 답 6 (분석메뉴 그리기 메뉴 추가) - grok

그리기 메뉴와 분석 메뉴를 추가해서 여러 기능들을 만들어줘 간단한 기본적인 것 부터 재미난 기능들을 추가해서 통합시켜줘 

요청하신 대로 기존 코드에 **그리기 메뉴**와 **분석 메뉴**를 추가하여 다양한 기능을 통합한 이미지 처리 프로그램을 제공하겠습니다. 툴팁은 작고 간결하게 유지하고, 이미지 표시 크기는 이미 확대된 상태를 유지하며, 새로운 메뉴에는 기본적인 기능부터 재미있는 기능까지 포함하겠습니다. 모든 변경 사항을 하나의 완전한 코드로 통합하고, 한글 주석과 설명을 포함하여 직관적으로 이해할 수 있도록 하겠습니다.

### 추가된 기능
1. **그리기 메뉴**:
   - **선 그리기**: 마우스로 필터링된 이미지에 자유롭게 선을 그릴 수 있습니다.
   - **사각형 그리기**: 클릭한 중심점을 기준으로 사각형을 그립니다.
   - **텍스트 추가**: 사용자가 입력한 텍스트를 이미지에 추가합니다.
   - **랜덤 도형 효과**: 이미지에 무작위로 원, 사각형, 선을 추가하여 예술적인 효과를 만듭니다.

2. **분석 메뉴**:
   - **히스토그램 표시**: 이미지의 밝기 분포를 분석하여 히스토그램 데이터를 반환합니다.
   - **색상 통계**: 이미지의 평균 색상과 주요 색상을 계산합니다.
   - **객체 감지 (간단한)**: 이미지에서 주요 윤곽선을 감지하여 강조합니다.
   - **픽셀 정보**: 클릭한 지점의 픽셀 색상 정보를 표시합니다.

### 코드 통합
아래는 모든 기능을 통합한 수정된 코드입니다. 기존의 툴팁 최적화, 이미지 크기 증가, 색상 메뉴 등을 유지하고, 새로운 그리기 및 분석 메뉴를 추가했습니다.

```python
import sys
import os
import cv2
import numpy as np
from PIL import Image
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QSlider, QVBoxLayout,
    QMenuBar, QMenu, QAction, QFileDialog, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QFont, QColor

import image_tools  # image_tools.load_image가 RGB 이미지를 반환한다고 가정

class ImageFilterWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("향상된 이미지 필터 프로그램")
        self.setGeometry(100, 100, 1400, 800)  Coaches 창 크기 증가

        self.original_image = None
        self.filtered_image = None
        self.current_filter = "Mean"
        self.click_point = None
        self.drawing = False
        self.last_point = None
        self.draw_mode = None

        # 툴팁 스타일 설정 (작고 깔끔하게)
        self.setStyleSheet("""
            QToolTip {
                font-size: 12px;
                padding: 4px;
                background-color: #333;
                color: white;
                border: 1px solid #666;
                border-radius: 4px;
            }
        """)

        self.create_menu_bar()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 설명 라벨
        self.info_label = QLabel("사진을 클릭하여 중심점을 설정하거나 그리기 동작을 수행하세요.")
        self.info_label.setToolTip("필터링된 이미지에서 클릭하여 중심점을 지정하거나 그림을 그립니다.")
        self.coord_label = QLabel("중심점 좌표: (자동 중앙)")
        self.main_layout.addWidget(self.info_label)
        self.main_layout.addWidget(self.coord_label)

        # 이미지 표시 (크기 증가)
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(600, 500)
        self.filtered_label = QLabel("필터링된 이미지")
        self.filtered_label.setAlignment(Qt.AlignCenter)
        self.filtered_label.setMinimumSize(600, 500)
        self.filtered_label.mousePressEvent = self.set_click_point
        self.filtered_label.mouseMoveEvent = self.draw_on_image
        self.filtered_label.mouseReleaseEvent = self.stop_drawing
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.filtered_label)
        self.main_layout.addLayout(self.image_layout)

        # 필터 크기 슬라이더
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(51)
        self.slider.setValue(3)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(5)
        self.slider.setToolTip("필터 크기를 조정합니다.")
        self.slider.valueChanged.connect(self.update_filtered_image)
        self.main_layout.addWidget(QLabel("필터 크기"))
        self.main_layout.addWidget(self.slider)

        # 회전 각도 슬라이더
        self.angle_slider = QSlider(Qt.Horizontal)
        self.angle_slider.setMinimum(-180)
        self.angle_slider.setMaximum(180)
        self.angle_slider.setValue(0)
        self.angle_slider.setTickPosition(QSlider.TicksBelow)
        self.angle_slider.setTickInterval(15)
        self.angle_slider.setToolTip("이미지 회전 각도를 조정합니다.")
        self.angle_slider.valueChanged.connect(self.rotate_image_slider)
        self.main_layout.addWidget(QLabel("회전 각도"))
        self.main_layout.addWidget(self.angle_slider)

    def create_menu_bar(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("이미지 열기", self)
        open_action.setShortcut("Ctrl+O")
        open_action.setToolTip("이미지 파일을 엽니다.")
        open_action.triggered.connect(self.open_image)
        file_menu.addAction(open_action)

        # 필터 메뉴
        filter_menu = menubar.addMenu("필터")
        filters = [
            ("평균 필터", "Mean", "Ctrl+1", "이미지를 부드럽게 합니다."),
            ("가우시안 블러", "Gaussian", "Ctrl+2", "가우시안 분포로 부드럽게 합니다."),
            ("미디언 필터", "Median", "Ctrl+3", "노이즈를 효과적으로 제거합니다."),
            ("소벨 필터", "Sobel", "Ctrl+4", "에지를 강조합니다."),
            ("라플라시안 필터", "Laplacian", "Ctrl+5", "에지를 선명하게 합니다."),
            ("웨이브 필터", "Wave", "Ctrl+6", "파동 효과를 적용합니다.")
        ]
        for name, mode, shortcut, tooltip in filters:
            action = QAction(name, self)
            action.setShortcut(shortcut)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.set_filter)
            filter_menu.addAction(action)

        # 에지 검출 메뉴
        edge_menu = menubar.addMenu("에지 검출")
        edges = [
            ("캐니 엣지", "Canny", "캐니 알고리즘으로 에지를 검출합니다."),
            ("프리윗 엣지", "Prewitt", "프리윗 필터로 에지를 검출합니다."),
            ("로버츠 엣지", "Roberts", "로버츠 필터로 에지를 검출합니다.")
        ]
        for name, mode, tooltip in edges:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.apply_edge_filter)
            edge_menu.addAction(action)

        # 노이즈 메뉴
        noise_menu = menubar.addMenu("노이즈")
        noise_action = QAction("가우시안 노이즈 추가", self)
        noise_action.setToolTip("이미지에 가우시안 노이즈를 추가합니다.")
        noise_action.triggered.connect(self.add_gaussian_noise)
        noise_menu.addAction(noise_action)

        # 색상 메뉴
        color_menu = menubar.addMenu("색상")
        color_actions = [
            ("색상 반전", self.invert_colors, "이미지의 색상을 반전시킵니다."),
            ("그레이스케일", self.to_grayscale, "이미지를 흑백으로 변환합니다."),
            ("세피아 톤", self.apply_sepia, "이미지에 세피아 톤을 적용합니다."),
            ("색조 조정", self.adjust_hue, "이미지의 색조를 조정합니다.")
        ]
        for name, func, tooltip in color_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            color_menu.addAction(action)

        # 그리기 메뉴 (신규)
        draw_menu = menubar.addMenu("그리기")
        draw_actions = [
            ("선 그리기", self.enable_line_drawing, "마우스로 자유롭게 선을 그립니다."),
            ("사각형 그리기", self.draw_rectangle, "중심점을 기준으로 사각형을 그립니다."),
            ("텍스트 추가", self.add_text, "이미지에 텍스트를 추가합니다."),
            ("랜덤 도형 효과", self.random_shapes, "무작위 도형을 추가하여 예술적 효과를 만듭니다.")
        ]
        for name, func, tooltip in draw_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            draw_menu.addAction(action)

        # 분석 메뉴 (신규)
        analysis_menu = menubar.addMenu("분석")
        analysis_actions = [
            ("히스토그램 표시", self.show_histogram, "이미지의 밝기 분포를 분석합니다."),
            ("색상 통계", self.color_statistics, "이미지의 평균 및 주요 색상을 계산합니다."),
            ("객체 감지", self.detect_objects, "주요 윤곽선을 감지하여 표시합니다."),
            ("픽셀 정보", self.show_pixel_info, "클릭한 지점의 픽셀 색상 정보를 표시합니다.")
        ]
        for name, func, tooltip in analysis_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            analysis_menu.addAction(action)

    def open_image(self):
        filename, _ = QFileDialog.getOpenFileName(self, "이미지 열기", "", "이미지 파일 (*.png *.jpg *.jpeg *.bmp)")
        if filename:
            self.original_image = image_tools.load_image(filename)
            self.filtered_image = self.original_image.copy()
            self.update_filtered_image()
            self.update_display_images()

    def set_filter(self):
        self.current_filter = self.sender().data()
        self.update_filtered_image()

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def set_click_point(self, event):
        if self.filtered_image is None:
            return
        x = event.pos().x()
        y = event.pos().y()
        label_width = self.filtered_label.width()
        label_height = self.filtered_label.height()
        img_height, img_width = self.filtered_image.shape[:2]
        scale_x = img_width / label_width
        scale_y = img_height / label_height
        self.click_point = (int(x * scale_x), int(y * scale_y))
        self.coord_label.setText(f"중심점 좌표: {self.click_point}")
        if self.draw_mode == "line":
            self.last_point = self.click_point
            self.drawing = True
        elif self.draw_mode == "rectangle":
            self.draw_rectangle()
        elif self.draw_mode == "pixel_info":
            self.show_pixel_info()
        self.rotate_image_slider()

    def draw_on_image(self, event):
        if self.drawing and self.draw_mode == "line" and self.filtered_image is not None:
            x = event.pos().x()
            y = event.pos().y()
            label_width = self.filtered_label.width()
            label_height = self.filtered_label.height()
            img_height, img_width = self.filtered_image.shape[:2]
            scale_x = img_width / label_width
            scale_y = img_height / label_height
            current_point = (int(x * scale_x), int(y * scale_y))
            img = self.filtered_image.copy()
            cv2.line(img, self.last_point, current_point, (255, 0, 0), 2)
            self.filtered_image = img
            self.last_point = current_point
            self.update_display_images()

    def stop_drawing(self, event):
        self.drawing = False
        self.last_point = None

    def enable_line_drawing(self):
        self.draw_mode = "line"
        self.info_label.setText("마우스를 드래그하여 선을 그립니다.")

    def draw_rectangle(self):
        if self.filtered_image is None or self.click_point is None:
            return
        img = self.filtered_image.copy()
        center = self.click_point
        size = 100  # 고정 크기 사각형
        top_left = (center[0] - size // 2, center[1] - size // 2)
        bottom_right = (center[0] + size // 2, center[1] + size // 2)
        cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)
        self.filtered_image = img
        self.draw_mode = None
        self.update_display_images()

    def add_text(self):
        if self.filtered_image is None:
            return
        text, ok = QInputDialog.getText(self, "텍스트 입력", "이미지에 추가할 텍스트를 입력하세요:")
        if ok and text:
            img = self.filtered_image.copy()
            pos = self.click_point if self.click_point else (50, 50)
            cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            self.filtered_image = img
            self.update_display_images()

    def random_shapes(self):
        if self.filtered_image is None:
            return
        img = self.filtered_image.copy()
        for _ in range(10):
            shape_type = np.random.choice(["circle", "rectangle", "line"])
            color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
            if shape_type == "circle":
                center = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                radius = np.random.randint(10, 50)
                cv2.circle(img, center, radius, color, 2)
            elif shape_type == "rectangle":
                top_left = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                bottom_right = (top_left[0] + 50, top_left[1] + 50)
                cv2.rectangle(img, top_left, bottom_right, color, 2)
            elif shape_type == "line":
                pt1 = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                pt2 = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                cv2.line(img, pt1, pt2, color, 2)
        self.filtered_image = img
        self.update_display_images()

    def show_histogram(self):
        if self.filtered_image is None:
            return
        gray = cv2.cvtColor(self.filtered_image, cv2.COLOR_RGB2GRAY)
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = hist.flatten()
        hist_info = f"히스토그램 데이터 (최대 10개 빈): {hist[:10]}"
        self.info_label.setText(hist_info)

    def color_statistics(self):
        if self.filtered_image is None:
            return
        mean_color = np.mean(self.filtered_image, axis=(0, 1))
        dominant_color = np.median(self.filtered_image.reshape(-1, 3), axis=0)
        stats = f"평균 색상 (RGB): {mean_color.astype(int)}, 주요 색상: {dominant_color.astype(int)}"
        self.info_label.setText(stats)

    def detect_objects(self):
        if self.filtered_image is None:
            return
        gray = cv2.cvtColor(self.filtered_image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img = self.filtered_image.copy()
        cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
        self.filtered_image = img
        self.update_display_images()

    def show_pixel_info(self):
        if self.filtered_image is None or self.click_point is None:
            return
        self.draw_mode = "pixel_info"
        x, y = self.click_point
        if 0 <= y < self.filtered_image.shape[0] and 0 <= x < self.filtered_image.shape[1]:
            pixel = self.filtered_image[y, x]
            self.info_label.setText(f"픽셀 ({x}, {y}) RGB: {pixel}")
        self.draw_mode = None

    def apply_wave_filter(self, image):
        img_pil = Image.fromarray(image).convert('L').resize((800, 800))
        width, height = img_pil.size
        pixels = np.array(img_pil)
        pixmap = QPixmap(width, height)
        pixmap.fill(Qt.white)
        painter = QPainter(pixmap)
        pen = QPen(Qt.black, 1)
        painter.setPen(pen)
        frequency = 0.5
        max_amplitude = 5
        num_lines = 100
        line_spacing = height // num_lines
        all_points = []
        for line_idx in range(num_lines):
            y_base = line_idx * line_spacing
            points = []
            for x in range(width):
                y_pixel = min(max(y_base, 0), height - 1)
                brightness = pixels[y_pixel, x] / 255.0
                amplitude = (1 - brightness) * max_amplitude
                wave_offset = amplitude * np.sin(frequency * x)
                y_pos = y_base + wave_offset
                points.append(QPoint(x, int(y_pos)))
            all_points.append(points)
        for i in range(len(all_points) - 1):
            current_points = all_points[i]
            next_points = all_points[i + 1]
            painter.drawLine(current_points[-1], next_points[-1]) if i % 2 == 0 else painter.drawLine(current_points[0], next_points[0])
            for j in range(len(current_points) - 1):
                painter.drawLine(current_points[j], current_points[j + 1])
        painter.end()
        qimage = pixmap.toImage().convertToFormat(QImage.Format_RGB888)
        width, height = qimage.width(), qimage.height()
        ptr = qimage.bits()
        ptr.setsize(height * width * 3)
        arr = np.frombuffer(ptr.asstring(), np.uint8).reshape((height, width, 3))
        return arr

    def apply_edge_filter(self):
        if self.original_image is None:
            return
        mode = self.sender().data()
        gray = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        if mode == "Canny":
            edges = cv2.Canny(gray, 100, 200)
        elif mode == "Prewitt":
            kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]], dtype=np.float32)
            kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        elif mode == "Roberts":
            kernelx = np.array([[1,0],[0,-1]], dtype=np.float32)
            kernely = np.array([[0,1],[-1,0]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        self.filtered_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def add_gaussian_noise(self):
        if self.original_image is None:
            return
        row, col, ch = self.original_image.shape
        gauss = np.random.normal(0, 25, (row, col, ch)).astype(np.float32)
        noisy = np.clip(self.original_image + gauss, 0, 255).astype(np.uint8)
        self.filtered_image = noisy
        self.update_display_images()

    def invert_colors(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.bitwise_not(self.original_image)
        self.update_display_images()

    def to_grayscale(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def apply_sepia(self):
        if self.original_image is None:
            return
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        sepia_matrix = np.array([[0.272, 0.534, 0.131],
                                 [0.349, 0.686, 0.168],
                                 [0.393, 0.769, 0.189]])
        self.filtered_image = cv2.transform(img_bgr, sepia_matrix)
        self.filtered_image = np.clip(self.filtered_image, 0, 255).astype(np.uint8)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        self.update_display_images()

    def adjust_hue(self):
        if self.original_image is None:
            return
        img_hsv = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2HSV)
        img_hsv[:, :, 0] = (img_hsv[:, :, 0].astype(np.float32) + 30) % 180
        self.filtered_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
        self.update_display_images()

    def rotate_image_slider(self):
        if self.original_image is None:
            return
        angle = self.angle_slider.value()
        center = self.click_point if self.click_point else (self.original_image.shape[1] // 2, self.original_image.shape[0] // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.original_image, M, (self.original_image.shape[1], self.original_image.shape[0]))
        self.filtered_image = rotated
        self.update_display_images()

    def update_filtered_image(self):
        if self.original_image is None:
            return
        kernel_size = self.slider.value()
        if self.current_filter not in ["Sobel", "Laplacian", "Wave"]:
            kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
            kernel_size = max(1, kernel_size)
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        if self.current_filter == "Mean":
            self.filtered_image = cv2.blur(img_bgr, (kernel_size, kernel_size))
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Gaussian":
            self.filtered_image = cv2.GaussianBlur(img_bgr, (kernel_size, kernel_size), 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Median":
            self.filtered_image = cv2.medianBlur(img_bgr, kernel_size)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Sobel":
            sobel_x = cv2.Sobel(img_bgr, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(img_bgr, cv2.CV_64F, 0, 1, ksize=3)
            sobel_x = cv2.convertScaleAbs(sobel_x)
            sobel_y = cv2.convertScaleAbs(sobel_y)
            self.filtered_image = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Laplacian":
            self.filtered_image = cv2.Laplacian(img_bgr, cv2.CV_64F, ksize=3)
            self.filtered_image = cv2.convertScaleAbs(self.filtered_image)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Wave":
            self.filtered_image = self.apply_wave_filter(self.original_image)
        self.update_display_images()

    def update_display_images(self):
        if self.original_image is None:
            return
        self.original_label.setPixmap(self.numpy_to_qpixmap(self.original_image).scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        if self.filtered_image is not None:
            self.filtered_label.setPixmap(self.numpy_to_qpixmap(self.filtered_image).scaled(
                self.filtered_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def moving_average_filtering(filename=None):
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        app.quit()
        app = QApplication(sys.argv)
    window = ImageFilterWindow()
    if filename:
        if os.path.exists(filename):
            window.original_image = image_tools.load_image(filename)
            window.filtered_image = window.original_image.copy()
            window.update_filtered_image()
            window.update_display_images()
    window.show()
    try:
        app.exec_()
    except SystemExit:
        pass

if __name__ == "__main__":
    moving_average_filtering()
```

### 주요 변경 사항
1. **그리기 메뉴**:
   - **선 그리기**: 마우스 드래그로 필터링된 이미지에 빨간색 선을 그립니다. `mouseMoveEvent`와 `mouseReleaseEvent`를 추가하여 실시간 드로잉을 구현했습니다.
   - **사각형 그리기**: 클릭한 중심점을 기준으로 고정 크기(100x100)의 녹색 사각형을 그립니다.
   - **텍스트 추가**: 사용자가 입력한 텍스트를 이미지에 빨간색으로 추가합니다. 위치는 클릭한 중심점 또는 기본 위치(50,50)를 사용합니다.
   - **랜덤 도형 효과**: 무작위로 원, 사각형, 선을 다양한 색상으로 10개 추가하여 예술적인 효과를 만듭니다.

2. **분석 메뉴**:
   - **히스토그램 표시**: 흑백 이미지의 밝기 히스토그램을 계산하여 상위 10개 빈 값을 정보 라벨에 표시합니다.
   - **색상 통계**: 이미지의 평균 색상과 주요 색상(중앙값)을 계산하여 RGB 값으로 표시합니다.
   - **객체 감지**: 캐니 에지 검출 후 윤곽선을 찾아 녹색으로 표시합니다. 간단한 객체 감지 효과를 제공합니다.
   - **픽셀 정보**: 클릭한 지점의 RGB 색상 값을 정보 라벨에 표시합니다.

3. **기존 기능 유지**:
   - 툴팁은 작고 간결하게 유지되며, 모든 메뉴 항목과 슬라이더에 적절한 설명이 포함되었습니다.
   - 이미지 표시 영역은 `(600, 500)` 크기로 유지되어 큰 이미지를 효과적으로 표시합니다.
   - 색상 메뉴(색상 반전, 그레이스케일, 세피아 톤, 색조 조정), 필터, 에지 검출, 노이즈 추가, 회전 기능은 그대로 유지되었습니다.

4. **사용성 개선**:
   - `set_click_point` 메서드를 확장하여 그리기와 픽셀 정보 표시를 지원하도록 수정했습니다.
   - 정보 라벨을 활용하여 분석 결과를 표시하고, 그리기 모드 변경 시 사용자에게 현재 상태를 안내합니다.
   - 모든 기능이 원활히 동작하도록 통합 테스트를 고려하여 코드를 구성했습니다.

### 사용 방법
- **이미지 열기**: "파일 > 이미지 열기"로 이미지를 로드합니다.
- **필터 적용**: "필터" 메뉴에서 원하는 필터를 선택하고, 필터 크기 슬라이더로 강도를 조정합니다.
- **에지 검출**: "에지 검출" 메뉴에서 캐니, 프리윗, 로버츠 중 선택합니다.
- **노이즈 추가**: "노이즈 > 가우시안 노이즈 추가"로 노이즈를 적용합니다.
- **색상 조정**: "색상" 메뉴에서 색상 반전, 그레이스케일, 세피아 톤, 색조 조정을 선택합니다.
- **그리기**: "그리기" 메뉴에서 선, 사각형, 텍스트, 랜덤 도형을 선택하여 이미지를 편집합니다.
- **분석**: "분석" 메뉴에서 히스토그램, 색상 통계, 객체 감지, 픽셀 정보를 확인합니다.
- **회전**: 중심점을 클릭한 후 회전 각도 슬라이더로 이미지를 회전합니다.

이 코드는 `image_tools.load_image` 함수가 RGB 이미지를 반환한다고 가정하며, 별도로 정의되어 있어야 합니다. 모든 기능은 통합되어 있으며, 직관적이고 재미있는 이미지 처리 경험을 제공합니다.

# 질문과 답 7 (오류 수정) - grok

Cell In[3],   line 19
    self.setGeometry(100, 100, 1400, 800)  Coaches 창 크기 증가
                                           ^
SyntaxError: invalid syntax
라는 에러가 나

에러 메시지를 보니 `self.setGeometry(100, 100, 1400, 800)  Coaches 창 크기 증가` 라인에서 구문 오류(SyntaxError)가 발생했습니다. 이는 코드 끝에 붙은 `Coaches 창 크기 증가`라는 주석이 잘못된 형식으로 작성되어 Python 파서가 이를 인식하지 못했기 때문입니다. Python에서는 주석이 `#`으로 시작해야 하며, 해당 라인에 `#`이 누락된 것으로 보입니다.

### 문제 원인
- `self.setGeometry(100, 100, 1400, 800)  Coaches 창 크기 증가`에서 `Coaches 창 크기 증가`는 주석이어야 하지만 `#` 없이 작성되어 구문 오류가 발생했습니다.
- 올바른 주석 형식은 `# Coaches 창 크기 증가`처럼 `#`로 시작해야 합니다.

### 수정 방법
해당 라인을 다음과 같이 수정하여 주석에 `#`를 추가합니다:
```python
self.setGeometry(100, 100, 1400, 800)  # 창 크기 증가
```

### 전체 코드 수정
아래는 에러를 수정한 전체 코드입니다. `Coaches 창 크기 증가`를 `# 창 크기 증가`로 수정하고, 나머지 코드는 그대로 유지했습니다. 또한, 코드의 일관성을 위해 주석을 간결하게 정리했습니다.

```python
```python
import sys
import os
import cv2
import numpy as np
from PIL import Image
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QSlider, QVBoxLayout,
    QMenuBar, QMenu, QAction, QFileDialog, QInputDialog
)
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QFont, QColor

import image_tools  # image_tools.load_image가 RGB 이미지를 반환한다고 가정

class ImageFilterWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("향상된 이미지 필터 프로그램")
        self.setGeometry(100, 100, 1400, 800)  # 창 크기 증가

        self.original_image = None
        self.filtered_image = None
        self.current_filter = "Mean"
        self.click_point = None
        self.drawing = False
        self.last_point = None
        self.draw_mode = None

        # 툴팁 스타일 설정 (작고 깔끔하게)
        self.setStyleSheet("""
            QToolTip {
                font-size: 12px;
                padding: 4px;
                background-color: #333;
                color: white;
                border: 1px solid #666;
                border-radius: 4px;
            }
        """)

        self.create_menu_bar()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 설명 라벨
        self.info_label = QLabel("사진을 클릭하여 중심점을 설정하거나 그리기 동작을 수행하세요.")
        self.info_label.setToolTip("필터링된 이미지에서 클릭하여 중심점을 지정하거나 그림을 그립니다.")
        self.coord_label = QLabel("중심점 좌표: (자동 중앙)")
        self.main_layout.addWidget(self.info_label)
        self.main_layout.addWidget(self.coord_label)

        # 이미지 표시 (크기 증가)
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setMinimumSize(600, 500)
        self.filtered_label = QLabel("필터링된 이미지")
        self.filtered_label.setAlignment(Qt.AlignCenter)
        self.filtered_label.setMinimumSize(600, 500)
        self.filtered_label.mousePressEvent = self.set_click_point
        self.filtered_label.mouseMoveEvent = self.draw_on_image
        self.filtered_label.mouseReleaseEvent = self.stop_drawing
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.filtered_label)
        self.main_layout.addLayout(self.image_layout)

        # 필터 크기 슬라이더
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(51)
        self.slider.setValue(3)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(5)
        self.slider.setToolTip("필터 크기를 조정합니다.")
        self.slider.valueChanged.connect(self.update_filtered_image)
        self.main_layout.addWidget(QLabel("필터 크기"))
        self.main_layout.addWidget(self.slider)

        # 회전 각도 슬라이더
        self.angle_slider = QSlider(Qt.Horizontal)
        self.angle_slider.setMinimum(-180)
        self.angle_slider.setMaximum(180)
        self.angle_slider.setValue(0)
        self.angle_slider.setTickPosition(QSlider.TicksBelow)
        self.angle_slider.setTickInterval(15)
        self.angle_slider.setToolTip("이미지 회전 각도를 조정합니다.")
        self.angle_slider.valueChanged.connect(self.rotate_image_slider)
        self.main_layout.addWidget(QLabel("회전 각도"))
        self.main_layout.addWidget(self.angle_slider)

    def create_menu_bar(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("이미지 열기", self)
        open_action.setShortcut("Ctrl+O")
        open_action.setToolTip("이미지 파일을 엽니다.")
        open_action.triggered.connect(self.open_image)
        file_menu.addAction(open_action)

        # 필터 메뉴
        filter_menu = menubar.addMenu("필터")
        filters = [
            ("평균 필터", "Mean", "Ctrl+1", "이미지를 부드럽게 합니다."),
            ("가우시안 블러", "Gaussian", "Ctrl+2", "가우시안 분포로 부드럽게 합니다."),
            ("미디언 필터", "Median", "Ctrl+3", "노이즈를 효과적으로 제거합니다."),
            ("소벨 필터", "Sobel", "Ctrl+4", "에지를 강조합니다."),
            ("라플라시안 필터", "Laplacian", "Ctrl+5", "에지를 선명하게 합니다."),
            ("웨이브 필터", "Wave", "Ctrl+6", "파동 효과를 적용합니다.")
        ]
        for name, mode, shortcut, tooltip in filters:
            action = QAction(name, self)
            action.setShortcut(shortcut)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.set_filter)
            filter_menu.addAction(action)

        # 에지 검출 메뉴
        edge_menu = menubar.addMenu("에지 검출")
        edges = [
            ("캐니 엣지", "Canny", "캐니 알고리즘으로 에지를 검출합니다."),
            ("프리윗 엣지", "Prewitt", "프리윗 필터로 에지를 검출합니다."),
            ("로버츠 엣지", "Roberts", "로버츠 필터로 에지를 검출합니다.")
        ]
        for name, mode, tooltip in edges:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.setData(mode)
            action.triggered.connect(self.apply_edge_filter)
            edge_menu.addAction(action)

        # 노이즈 메뉴
        noise_menu = menubar.addMenu("노이즈")
        noise_action = QAction("가우시안 노이즈 추가", self)
        noise_action.setToolTip("이미지에 가우시안 노이즈를 추가합니다.")
        noise_action.triggered.connect(self.add_gaussian_noise)
        noise_menu.addAction(noise_action)

        # 색상 메뉴
        color_menu = menubar.addMenu("색상")
        color_actions = [
            ("색상 반전", self.invert_colors, "이미지의 색상을 반전시킵니다."),
            ("그레이스케일", self.to_grayscale, "이미지를 흑백으로 변환합니다."),
            ("세피아 톤", self.apply_sepia, "이미지에 세피아 톤을 적용합니다."),
            ("색조 조정", self.adjust_hue, "이미지의 색조를 조정합니다.")
        ]
        for name, func, tooltip in color_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            color_menu.addAction(action)

        # 그리기 메뉴
        draw_menu = menubar.addMenu("그리기")
        draw_actions = [
            ("선 그리기", self.enable_line_drawing, "마우스로 자유롭게 선을 그립니다."),
            ("사각형 그리기", self.draw_rectangle, "중심점을 기준으로 사각형을 그립니다."),
            ("텍스트 추가", self.add_text, "이미지에 텍스트를 추가합니다."),
            ("랜덤 도형 효과", self.random_shapes, "무작위 도형을 추가하여 예술적 효과를 만듭니다.")
        ]
        for name, func, tooltip in draw_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            draw_menu.addAction(action)

        # 분석 메뉴
        analysis_menu = menubar.addMenu("분석")
        analysis_actions = [
            ("히스토그램 표시", self.show_histogram, "이미지의 밝기 분포를 분석합니다."),
            ("색상 통계", self.color_statistics, "이미지의 평균 및 주요 색상을 계산합니다."),
            ("객체 감지", self.detect_objects, "주요 윤곽선을 감지하여 표시합니다."),
            ("픽셀 정보", self.show_pixel_info, "클릭한 지점의 픽셀 색상 정보를 표시합니다.")
        ]
        for name, func, tooltip in analysis_actions:
            action = QAction(name, self)
            action.setToolTip(tooltip)
            action.triggered.connect(func)
            analysis_menu.addAction(action)

    def open_image(self):
        filename, _ = QFileDialog.getOpenFileName(self, "이미지 열기", "", "이미지 파일 (*.png *.jpg *.jpeg *.bmp)")
        if filename:
            self.original_image = image_tools.load_image(filename)
            self.filtered_image = self.original_image.copy()
            self.update_filtered_image()
            self.update_display_images()

    def set_filter(self):
        self.current_filter = self.sender().data()
        self.update_filtered_image()

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def set_click_point(self, event):
        if self.filtered_image is None:
            return
        x = event.pos().x()
        y = event.pos().y()
        label_width = self.filtered_label.width()
        label_height = self.filtered_label.height()
        img_height, img_width = self.filtered_image.shape[:2]
        scale_x = img_width / label_width
        scale_y = img_height / label_height
        self.click_point = (int(x * scale_x), int(y * scale_y))
        self.coord_label.setText(f"중심점 좌표: {self.click_point}")
        if self.draw_mode == "line":
            self.last_point = self.click_point
            self.drawing = True
        elif self.draw_mode == "rectangle":
            self.draw_rectangle()
        elif self.draw_mode == "pixel_info":
            self.show_pixel_info()
        self.rotate_image_slider()

    def draw_on_image(self, event):
        if self.drawing and self.draw_mode == "line" and self.filtered_image is not None:
            x = event.pos().x()
            y = event.pos().y()
            label_width = self.filtered_label.width()
            label_height = self.filtered_label.height()
            img_height, img_width = self.filtered_image.shape[:2]
            scale_x = img_width / label_width
            scale_y = img_height / label_height
            current_point = (int(x * scale_x), int(y * scale_y))
            img = self.filtered_image.copy()
            cv2.line(img, self.last_point, current_point, (255, 0, 0), 2)
            self.filtered_image = img
            self.last_point = current_point
            self.update_display_images()

    def stop_drawing(self, event):
        self.drawing = False
        self.last_point = None

    def enable_line_drawing(self):
        self.draw_mode = "line"
        self.info_label.setText("마우스를 드래그하여 선을 그립니다.")

    def draw_rectangle(self):
        if self.filtered_image is None or self.click_point is None:
            return
        img = self.filtered_image.copy()
        center = self.click_point
        size = 100
        top_left = (center[0] - size // 2, center[1] - size // 2)
        bottom_right = (center[0] + size // 2, center[1] + size // 2)
        cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 2)
        self.filtered_image = img
        self.draw_mode = None
        self.update_display_images()

    def add_text(self):
        if self.filtered_image is None:
            return
        text, ok = QInputDialog.getText(self, "텍스트 입력", "이미지에 추가할 텍스트를 입력하세요:")
        if ok and text:
            img = self.filtered_image.copy()
            pos = self.click_point if self.click_point else (50, 50)
            cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            self.filtered_image = img
            self.update_display_images()

    def random_shapes(self):
        if self.filtered_image is None:
            return
        img = self.filtered_image.copy()
        for _ in range(10):
            shape_type = np.random.choice(["circle", "rectangle", "line"])
            color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
            if shape_type == "circle":
                center = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                radius = np.random.randint(10, 50)
                cv2.circle(img, center, radius, color, 2)
            elif shape_type == "rectangle":
                top_left = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                bottom_right = (top_left[0] + 50, top_left[1] + 50)
                cv2.rectangle(img, top_left, bottom_right, color, 2)
            elif shape_type == "line":
                pt1 = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                pt2 = (np.random.randint(50, img.shape[1]-50), np.random.randint(50, img.shape[0]-50))
                cv2.line(img, pt1, pt2, color, 2)
        self.filtered_image = img
        self.update_display_images()

    def show_histogram(self):
        if self.filtered_image is None:
            return
        gray = cv2.cvtColor(self.filtered_image, cv2.COLOR_RGB2GRAY)
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        hist = hist.flatten()
        hist_info = f"히스토그램 데이터 (최대 10개 빈): {hist[:10]}"
        self.info_label.setText(hist_info)

    def color_statistics(self):
        if self.filtered_image is None:
            return
        mean_color = np.mean(self.filtered_image, axis=(0, 1))
        dominant_color = np.median(self.filtered_image.reshape(-1, 3), axis=0)
        stats = f"평균 색상 (RGB): {mean_color.astype(int)}, 주요 색상: {dominant_color.astype(int)}"
        self.info_label.setText(stats)

    def detect_objects(self):
        if self.filtered_image is None:
            return
        gray = cv2.cvtColor(self.filtered_image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img = self.filtered_image.copy()
        cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
        self.filtered_image = img
        self.update_display_images()

    def show_pixel_info(self):
        if self.filtered_image is None or self.click_point is None:
            return
        self.draw_mode = "pixel_info"
        x, y = self.click_point
        if 0 <= y < self.filtered_image.shape[0] and 0 <= x < self.filtered_image.shape[1]:
            pixel = self.filtered_image[y, x]
            self.info_label.setText(f"픽셀 ({x}, {y}) RGB: {pixel}")
        self.draw_mode = None

    def apply_wave_filter(self, image):
        img_pil = Image.fromarray(image).convert('L').resize((800, 800))
        width, height = img_pil.size
        pixels = np.array(img_pil)
        pixmap = QPixmap(width, height)
        pixmap.fill(Qt.white)
        painter = QPainter(pixmap)
        pen = QPen(Qt.black, 1)
        painter.setPen(pen)
        frequency = 0.5
        max_amplitude = 5
        num_lines = 100
        line_spacing = height // num_lines
        all_points = []
        for line_idx in range(num_lines):
            y_base = line_idx * line_spacing
            points = []
            for x in range(width):
                y_pixel = min(max(y_base, 0), height - 1)
                brightness = pixels[y_pixel, x] / 255.0
                amplitude = (1 - brightness) * max_amplitude
                wave_offset = amplitude * np.sin(frequency * x)
                y_pos = y_base + wave_offset
                points.append(QPoint(x, int(y_pos)))
            all_points.append(points)
        for i in range(len(all_points) - 1):
            current_points = all_points[i]
            next_points = all_points[i + 1]
            painter.drawLine(current_points[-1], next_points[-1]) if i % 2 == 0 else painter.drawLine(current_points[0], next_points[0])
            for j in range(len(current_points) - 1):
                painter.drawLine(current_points[j], current_points[j + 1])
        painter.end()
        qimage = pixmap.toImage().convertToFormat(QImage.Format_RGB888)
        width, height = qimage.width(), qimage.height()
        ptr = qimage.bits()
        ptr.setsize(height * width * 3)
        arr = np.frombuffer(ptr.asstring(), np.uint8).reshape((height, width, 3))
        return arr

    def apply_edge_filter(self):
        if self.original_image is None:
            return
        mode = self.sender().data()
        gray = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        if mode == "Canny":
            edges = cv2.Canny(gray, 100, 200)
        elif mode == "Prewitt":
            kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]], dtype=np.float32)
            kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        elif mode == "Roberts":
            kernelx = np.array([[1,0],[0,-1]], dtype=np.float32)
            kernely = np.array([[0,1],[-1,0]], dtype=np.float32)
            edges = cv2.filter2D(gray, -1, kernelx) + cv2.filter2D(gray, -1, kernely)
        self.filtered_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def add_gaussian_noise(self):
        if self.original_image is None:
            return
        row, col, ch = self.original_image.shape
        gauss = np.random.normal(0, 25, (row, col, ch)).astype(np.float32)
        noisy = np.clip(self.original_image + gauss, 0, 255).astype(np.uint8)
        self.filtered_image = noisy
        self.update_display_images()

    def invert_colors(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.bitwise_not(self.original_image)
        self.update_display_images()

    def to_grayscale(self):
        if self.original_image is None:
            return
        self.filtered_image = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_GRAY2RGB)
        self.update_display_images()

    def apply_sepia(self):
        if self.original_image is None:
            return
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        sepia_matrix = np.array([[0.272, 0.534, 0.131],
                                 [0.349, 0.686, 0.168],
                                 [0.393, 0.769, 0.189]])
        self.filtered_image = cv2.transform(img_bgr, sepia_matrix)
        self.filtered_image = np.clip(self.filtered_image, 0, 255).astype(np.uint8)
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        self.update_display_images()

    def adjust_hue(self):
        if self.original_image is None:
            return
        img_hsv = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2HSV)
        img_hsv[:, :, 0] = (img_hsv[:, :, 0].astype(np.float32) + 30) % 180
        self.filtered_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
        self.update_display_images()

    def rotate_image_slider(self):
        if self.original_image is None:
            return
        angle = self.angle_slider.value()
        center = self.click_point if self.click_point else (self.original_image.shape[1] // 2, self.original_image.shape[0] // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.original_image, M, (self.original_image.shape[1], self.original_image.shape[0]))
        self.filtered_image = rotated
        self.update_display_images()

    def update_filtered_image(self):
        if self.original_image is None:
            return
        kernel_size = self.slider.value()
        if self.current_filter not in ["Sobel", "Laplacian", "Wave"]:
            kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
            kernel_size = max(1, kernel_size)
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        if self.current_filter == "Mean":
            self.filtered_image = cv2.blur(img_bgr, (kernel_size, kernel_size))
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Gaussian":
            self.filtered_image = cv2.GaussianBlur(img_bgr, (kernel_size, kernel_size), 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Median":
            self.filtered_image = cv2.medianBlur(img_bgr, kernel_size)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Sobel":
            sobel_x = cv2.Sobel(img_bgr, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(img_bgr, cv2.CV_64F, 0, 1, ksize=3)
            sobel_x = cv2.convertScaleAbs(sobel_x)
            sobel_y = cv2.convertScaleAbs(sobel_y)
            self.filtered_image = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Laplacian":
            self.filtered_image = cv2.Laplacian(img_bgr, cv2.CV_64F, ksize=3)
            self.filtered_image = cv2.convertScaleAbs(self.filtered_image)
            self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)
        elif self.current_filter == "Wave":
            self.filtered_image = self.apply_wave_filter(self.original_image)
        self.update_display_images()

    def update_display_images(self):
        if self.original_image is None:
            return
        self.original_label.setPixmap(self.numpy_to_qpixmap(self.original_image).scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        if self.filtered_image is not None:
            self.filtered_label.setPixmap(self.numpy_to_qpixmap(self.filtered_image).scaled(
                self.filtered_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def moving_average_filtering(filename=None):
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        app.quit()
        app = QApplication(sys.argv)
    window = ImageFilterWindow()
    if filename:
        if os.path.exists(filename):
            window.original_image = image_tools.load_image(filename)
            window.filtered_image = window.original_image.copy()
            window.update_filtered_image()
            window.update_display_images()
    window.show()
    try:
        app.exec_()
    except SystemExit:
        pass

if __name__ == "__main__":
    moving_average_filtering()
```

### 변경 사항 요약
- **에러 수정**: `self.setGeometry(100, 100, 1400, 800)` 뒤의 잘못된 주석 `Coaches 창 크기 증가`를 `# 창 크기 증가`로 수정하여 구문 오류를 해결했습니다.
- **코드 유지**: 기존의 모든 기능(필터, 에지 검출, 노이즈 추가, 색상 조정, 그리기, 분석)은 그대로 유지했습니다.
- **주석 정리**: 주석을 간결하고 명확하게 정리하여 가독성을 높였습니다.

### 실행 방법
1. `image_tools.load_image` 함수가 RGB 이미지를 반환하도록 별도로 정의해야 합니다.
2. 위 코드를 `.py` 파일로 저장한 후 실행하면 됩니다.
3. 프로그램은 이미지를 로드하고, 필터링, 색상 조정, 그리기, 분석 기능을 제공합니다.

이제 코드가 올바르게 실행될 것입니다. 추가적인 문제가 발생하면 말씀해주세요!